In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q pydrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
!pip install transformers

In [ ]:
!pip install simpletransformers

In [ ]:
project_path='/content/drive/My Drive/Fake News/'

In [ ]:
import numpy as np
import pandas as pd

dataset = pd.read_csv(project_path+'fake_or_real_news.csv')
print(dataset.shape)

texts=[]
texts=dataset['text']#####################################
label=dataset['label']

from sklearn.preprocessing import LabelEncoder
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))


#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

training_size=int(0.8*dataset.shape[0])
print(dataset.shape[0],training_size)
data_train=dataset[:training_size]['text']
y_train_val=y[:training_size]
data_test=dataset[training_size:]['text']
y_test=y[training_size:]


In [ ]:
sentences = data_train
print(sentences)
labels = y_train_val[:,0]
#labels = y_train
print(labels)

In [ ]:
print('Max sentence length: ', max([len(sen.split(' ')) for sen in sentences]))

In [ ]:
print(sentences[:1])

In [ ]:
TRAIN_SPLIT = 0.8

nb_train_samples = int(TRAIN_SPLIT * sentences.shape[0])

x_train = np.array(sentences[:nb_train_samples])
y_train = np.array(labels[:nb_train_samples])
x_val = np.array(sentences[nb_train_samples:])
y_val = np.array(labels[nb_train_samples:])


In [ ]:
import pandas as pd
ll=[]
for i in range(len(x_train)):
  ll.append([x_train[i],y_train[i]])

train_df = pd.DataFrame(ll,columns=['text','labels'])


ll=[]
for i in range(len(x_val)):
  ll.append([x_val[i],y_val[i]])

val_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
sentences_test = np.array(data_test)
print(sentences_test[0])
labels_test = y_test[:, 0]
#labels = y_train
print(labels_test)

In [ ]:
ll=[]
for i in range(len(sentences_test)):
  ll.append([sentences_test[i],labels_test[i]])

test_df = pd.DataFrame(ll,columns=['text','labels'])

In [ ]:
print('Training sample: ' + str(len(train_df['text'])))
print('Validation sample: ' + str(len(val_df['text'])))
print('Testing sample: ' + str(len(test_df['text'])))

In [ ]:
model_type = "roberta"
model_name = "roberta-base"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": "/content/drive/My Drive/Fake News/Transformer/output/"+model_type,
    "best_model_dir": "/content/drive/My Drive/Fake News/Transformer/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 10,
    "evaluate_during_training_steps": 64,
    "wandb_project": "Fake News FR_1 RoBerta",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "train_batch_size": 64,
    "eval_batch_size": 64,
    "evaluate_during_training_verbose" : True
}

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
#import pandas as pd

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 32
train_args["gradient_accumulation_steps"] = 1
train_args["evaluate_during_training"] = True
train_args["num_train_epochs"] = 10
train_args["use_early_stopping"] = True


# Create a ClassificationModel
model = ClassificationModel(model_type, model_name, args=train_args)

# Train the model
model.train_model(train_df,eval_df = val_df)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [ ]:
predictions, raw_outputs = model.predict(test_df['text'])

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_df['labels'],predictions))

In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Roberta_1_FR.pickle', 'wb') as f:
  pickle.dump((test_df['labels'],predictions),f)


In [ ]:
import pickle
with open(project_path+'Predictions/pickle_Pred_Roberta_1_FR.pickle', 'rb') as f:
  y_true, y_pred = pickle.load(f)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))